<a href="https://colab.research.google.com/github/seeedata/BADA/blob/main/%EB%8C%80%ED%95%99%EC%9B%90%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%20%ED%81%AC%EB%A1%A4%EB%A7%81%20%EB%B3%B4%EC%A1%B0/Rotten%20Tomato%20%EC%98%81%ED%99%94%20%EC%A0%95%EB%B3%B4%20%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import pandas as pd
import numpy as np

# 데이터셋 불러오기

In [ ]:
movie = pd.read_csv('C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/영화크롤링최종.csv'); movie

,ranks,titles,index,gross,theaters,per_theater,total_gross
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690"
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981"
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009"
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042"
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715"
...,...,...,...,...,...,...,...
804,-,All My Life,359,"$23,840",133,$179,"$899,585"
805,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864"
806,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940"
807,-,Archenemy,359,"$11,003",2,"$5,502","$96,828"


In [ ]:
movie['titles'].unique()

array(['Star Wars: The Rise of Skywalker', 'Jumanji: The Next Level',
       'Little Women', 'Frozen II', 'The Grudge', 'Spies in Disguise',
       'Knives Out', 'Uncut Gems', 'Bombshell', 'Cats', 'Richard Jewell',
       'Ford v. Ferrari', 'A Beautiful Day in the Neighborhood',
       'Queen & Slim', 'Parasite', '1917', 'IP Man: The Finale',
       '21 Bridges', 'Jojo Rabbit', 'A Hidden Life', 'Just Mercy',
       'Like a Boss', 'Underwater', 'Weathering With You',
       'Bad Boys For Life', 'Dolittle', 'Once Upon a Time…in Hollywood',
       'The Gentlemen', 'The Turning', 'The Last Full Measure',
       'Gretel & Hansel', 'The Rhythm Section', '2020 Oscar Shorts',
       'Birds of Prey (And the Fantabulous Emancipation…',
       'Sonic The Hedgehog', 'Fantasy Island', 'The Photograph',
       'Downhill', 'The Call of the Wild', 'Brahms: The Boy II',
       'My Hero Academia: Heroes: Rising',
       'Impractical Jokers: The Movie', 'My Boyfriend’s Meds',
       'Portrait of a Lady o

unique 영화 제목만 있는 데이터프레임 만들고, 여기에 정보 이어 붙여서 토마토 데이터프레임 만들기
-> 영화 제목 기준으로 movie 데이터프레임과 합치기

In [ ]:
tomato = pd.DataFrame(movie['titles'].unique(), columns=['titles']); tomato

,titles
0,Star Wars: The Rise of Skywalker
1,Jumanji: The Next Level
2,Little Women
3,Frozen II
4,The Grudge
...,...
178,Hunter Hunter
179,Wonder Woman 1984
180,News of the World
181,Promising Young Woman


https://www.rottentomatoes.com/m/star_wars_the_rise_of_skywalker

1. Genre
: #info > li:nth-child(2) > p > span

2. Original Language
: #info > li:nth-child(3) > p > span

3. Director
: #info > li:nth-child(4) > p > span > a

4. Release Date (Theaters)
: #info > li:nth-child(7) > p > span > time

5. Release Date (Streaming)
: #info > li:nth-child(8) > p > span > time

6. RunTime
: #info > li:nth-child(10) > p > span > time

7. Distributor
: #info > li:nth-child(11) > p > span

8. Production Co
: #info > li:nth-child(12) > p > span

9. Box Office (Gross USA)
: #info > li:nth-child(9) > p > span

In [ ]:
a = tomato.loc[1].values[0]; a

'Jumanji: The Next Level'

In [ ]:
a.lower()

'jumanji: the next level'

In [ ]:
a.replace('...', '')

'Jumanji: The Next Level'

In [ ]:
#tomato 데이터프레임에서 제목 하나를 입력으로 넣어서, 제목을 넣은 url을 만들어주고, 거기로 들어가서 정보 크롤링해주는 함수

def crawling(tomato, row):

    title = tomato.loc[row].values[0]  #영화 제목 저장

    #소문자로 변환 -> 특수문자 제거 -> 공백을 _로 변환
    title = title.lower().replace(':', '').replace('...', '').replace('&', 'and').replace("'", '').replace('-', '').replace('.', '').replace('Re-Release', '').replace(' ', '_')

    url = f'https://www.rottentomatoes.com/m/{title}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Whale/3.23.214.10 Safari/537.36'}

    try:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "lxml")

        info = soup.select("#info")
        info_str = info[0].text if info else None

        tomato.at[row, 'info'] = info_str

        time.sleep(1.5)

    except Exception as e:
        print(f"An error occurred: {e}")
        print(f'{row}')

    return tomato

In [ ]:
for row in tqdm(range(len(tomato))):
    crawling(tomato, row)
tomato

  0%|          | 0/183 [00:00<?, ?it/s]

,titles,info
0,Star Wars: The Rise of Skywalker,\n\n\nRating:\nPG-13 (Sci-Fi Violence and Acti...
1,Jumanji: The Next Level,\n\n\nRating:\nPG-13 (Some Language|Adventure ...
2,Little Women,\n\n\nGenre:\n\n \n...
3,Frozen II,\n\n\nRating:\nPG (Some Thematic Elements|Acti...
4,The Grudge,\n\n\nRating:\nPG-13 (Mature Thematic Material...
...,...,...
178,Hunter Hunter,\n\n\nRating:\nR\n \n\n...
179,Wonder Woman 1984,\n\n\nRating:\nPG-13 (Violence|Sequences of Ac...
180,News of the World,\n\n\nRating:\nPG-13 (Some Language|Disturbing...
181,Promising Young Woman,\n\n\nRating:\nR (Language Throughout|Drug Use...


In [ ]:
tomato

,titles,info
0,Star Wars: The Rise of Skywalker,\n\n\nRating:\nPG-13 (Sci-Fi Violence and Acti...
1,Jumanji: The Next Level,\n\n\nRating:\nPG-13 (Some Language|Adventure ...
2,Little Women,\n\n\nGenre:\n\n \n...
3,Frozen II,\n\n\nRating:\nPG (Some Thematic Elements|Acti...
4,The Grudge,\n\n\nRating:\nPG-13 (Mature Thematic Material...
...,...,...
178,Hunter Hunter,\n\n\nRating:\nR\n \n\n...
179,Wonder Woman 1984,\n\n\nRating:\nPG-13 (Violence|Sequences of Ac...
180,News of the World,\n\n\nRating:\nPG-13 (Some Language|Disturbing...
181,Promising Young Woman,\n\n\nRating:\nR (Language Throughout|Drug Use...


# 크롤링 안 된 영화 파악

In [ ]:
tomato[tomato['info'].isna()]

,titles,info
15,1917,None
16,IP Man: The Finale,None
26,Once Upon a Time…in Hollywood,None
32,2020 Oscar Shorts,None
33,Birds of Prey (And the Fantabulous Emancipation…,None
42,My Boyfriend’s Meds,None
46,Emma.,None
54,"Phoenix, Oregon",None
55,Be Natural: The Untold Story Of Alice Guy-Blaché,None
59,Best of Catvideofest,None


In [ ]:
len(tomato[tomato['info'].isna()])

25

In [ ]:
len(noinfo_url)

25

In [ ]:
noinfo_url = ['1917_2019',
              'ip_man_4_the_finale',
              'once_upon_a_time_in_hollywood',
              '2020_oscar_nominated_shorts_live_action',
              'birds_of_prey_2020',
              'my_boyfriends_meds',
              'emma_2020',
              'phoenix_oregon',
              'be_natural_the_untold_story_of_alice_guy_blache',
              'Best of Catvideofest',  #None
              'Monero Means Money: Cryptocurrency 101, Live fr…',  #None
              'how_to_build_a_girl_2020',
              'Circles: The Restorative Justice Project',  #None
              'i_propose_we_never_see_each_other_again_after_tonight',
              'my_brothers_crossing',
              'rent_a_pal',
              'save_yourselves',
              'jiang_ziya',
              'fatman',
              'santa_clause',
              'true_to_the_game_2',
              'national_lampoons_christmas_vacation',
              'all_my_life_2020',
              'polar_express',
              'godfather_part_iii'
              ]

In [ ]:
noinfo = tomato[tomato['info'].isna()].reset_index(drop=True)

def crawling(noinfo, row):

    url = f'https://www.rottentomatoes.com/m/{noinfo_url[row]}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Whale/3.23.214.10 Safari/537.36'}

    try:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "lxml")

        info = soup.select(f"#info")
        info_str = info[0].text if info else None

        noinfo.at[row, 'info'] = info_str

        time.sleep(1.5)

    except Exception as e:
        print(f"An error occurred: {e}")
        print(f'{row}')

    return noinfo

for row in tqdm(range(len(noinfo))):
    crawling(noinfo, row)
noinfo

  0%|          | 0/25 [00:00<?, ?it/s]

,titles,info
0,1917,\n\n\nRating:\nR (Some Disturbing Images|Langu...
1,IP Man: The Finale,\n\n\nGenre:\n\n \n...
2,Once Upon a Time…in Hollywood,\n\n\nRating:\nR (Language Throughout|Drug Use...
3,2020 Oscar Shorts,\n\n\nGenre:\n\n \n...
4,Birds of Prey (And the Fantabulous Emancipation…,\n\n\nRating:\nR (Some Sexual and Drug Materia...
5,My Boyfriend’s Meds,\n\n\nRating:\nR (Drug Material|Sexual Content...
6,Emma.,\n\n\nRating:\nPG (Brief Partial Nudity)\n ...
7,"Phoenix, Oregon",\n\n\nRating:\nR (Language)\n ...
8,Be Natural: The Untold Story Of Alice Guy-Blaché,\n\n\nGenre:\n\n \n...
9,Best of Catvideofest,None


In [ ]:
#기존 데이터프레임과 합치기
tomato = pd.merge(tomato, noinfo, on='titles', how='outer')
tomato['info'] = tomato['info_x'].combine_first(tomato['info_y'])
tomato.drop(['info_x', 'info_y'], axis=1, inplace=True)

In [ ]:
tomato

,titles,info
0,Star Wars: The Rise of Skywalker,\n\n\nRating:\nPG-13 (Sci-Fi Violence and Acti...
1,Jumanji: The Next Level,\n\n\nRating:\nPG-13 (Some Language|Adventure ...
2,Little Women,\n\n\nGenre:\n\n \n...
3,Frozen II,\n\n\nRating:\nPG (Some Thematic Elements|Acti...
4,The Grudge,\n\n\nRating:\nPG-13 (Mature Thematic Material...
...,...,...
178,Hunter Hunter,\n\n\nRating:\nR\n \n\n...
179,Wonder Woman 1984,\n\n\nRating:\nPG-13 (Violence|Sequences of Ac...
180,News of the World,\n\n\nRating:\nPG-13 (Some Language|Disturbing...
181,Promising Young Woman,\n\n\nRating:\nR (Language Throughout|Drug Use...


In [ ]:
tomato[tomato['info'].isna()]

,titles,info
59,Best of Catvideofest,None
60,"Monero Means Money: Cryptocurrency 101, Live fr…",None
69,Circles: The Restorative Justice Project,None


In [ ]:
tomato.to_csv('C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/토마토크롤링_모든info크롤링.csv')

# info slicing

In [ ]:
#tomato = pd.read_csv('C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/토마토크롤링_모든info크롤링.csv', index_col=0); tomato

,titles,info
0,Star Wars: The Rise of Skywalker,\n\n\nRating:\nPG-13 (Sci-Fi Violence and Acti...
1,Jumanji: The Next Level,\n\n\nRating:\nPG-13 (Some Language|Adventure ...
2,Little Women,\n\n\nGenre:\n\n \n...
3,Frozen II,\n\n\nRating:\nPG (Some Thematic Elements|Acti...
4,The Grudge,\n\n\nRating:\nPG-13 (Mature Thematic Material...
...,...,...
338,St. Patrick’s Day,NaN
339,Wonder Woman 1984,\n\n\nRating:\nPG-13 (Violence|Sequences of Ac...
340,News of the World,\n\n\nRating:\nPG-13 (Some Language|Disturbing...
341,Promising Young Woman,\n\n\nRating:\nR (Language Throughout|Drug Use...


In [ ]:
tomato['info'][0]

'\n\n\nRating:\nPG-13 (Sci-Fi Violence and Action)\n                        \n\n\n\n\nGenre:\n\n                            \n                            Sci-fi, \n                            \n                            Adventure, \n                            \n                            Action, \n                            \n                            Fantasy\n                            \n                        \n\n\n\n\nOriginal Language:\nEnglish\n\n\n\n\nDirector:\n\nJ.J. Abrams\n\n\n\n\n\nProducer:\n\nKathleen Kennedy, \n                                \n                            \n                                \n                                    J.J. Abrams, \n                                \n                            \n                                \n                                    Michelle Rejwan\n\n\n\n\n\nWriter:\n\nChris Terrio, \n                                \n                            \n                                \n                         

In [ ]:
tomato['info'][51]

'\n\n\nRating:\nPG (Thematic Material)\n                        \n\n\n\n\nGenre:\n\n                            \n                            Biography, \n                            \n                            Drama, \n                            \n                            Romance, \n                            \n                            Music\n                            \n                        \n\n\n\n\nOriginal Language:\nEnglish\n\n\n\n\nDirector:\n\nJon Erwin, \n                                \n                            \n                                \n                                    Andrew Erwin\n\n\n\n\n\nProducer:\n\nKevin Downes\n\n\n\n\n\nWriter:\n\nJon Erwin, \n                                \n                            \n                                \n                                    Jon Gunn\n\n\n\n\n\nRelease Date (Theaters):\n\nMar 13, 2020\n\xa0wide\n\n\n\n\n\nRelease Date (Streaming):\n\n\n                                Mar 26, 2020\n     

In [ ]:
genres = list()
ols = list()
ds = list()
rds = list()
runs = list()
dists = list()
pcs = list()
boxs = list()

def slicing(row, genres, ols, ds, rds, runs, dists, pcs, boxs):
    text = str(tomato.loc[row, 'info'])

    #Genre => 완료
    genre_start = text.find("Genre:")
    if genre_start != -1:
        genre_end = text.find("\n\n\n\n\n", genre_start)
        genre = text[genre_start + len("Genre:") : genre_end].replace('\n', '').replace('  ', '').strip()
        genres.append(genre)
    else:
        genre = np.nan
        genres.append(genre)

    #Originial Language => 완료
    ol_start = text.find('Original Language:')
    if ol_start != -1:
        ol_end = text.find('\n\n\n\n\n', ol_start)
        ol = text[ol_start + len('Original Language:') : ol_end].replace('\n', '').replace('  ', '').strip()
        ols.append(ol)
    else:
        ol = np.nan
        ols.append(ol)

    #director => 완료
    d_start = text.find('Director:')
    if d_start != -1:
        d_end = text.find('\n\n\n\n\n\n', d_start)
        d = text[d_start + len('Director:') : d_end].replace('\n', '').replace('  ', '').strip()
        ds.append(d)
    else:
        d = np.nan
        ds.append(d)

    #release date => 완료
    rd_start = text.find('Release Date (Theaters):')
    if rd_start != -1:
        rd_end = text.find('\n\n\n\n\n\n', rd_start)
        rd = text[rd_start + len('Release Date (Theaters):') : rd_end].replace('\n', '').replace('  ', '').replace('\xa0wide', '').replace('\xa0limited', '').replace('\xa0original', '').strip()
        rds.append(rd)
    else:
        rd = np.nan
        rds.append(rd)

    #runtime => 완료
    run_start = text.find('Runtime:')
    if run_start != -1:
        run_end = text.find('\n\n\n\n\n\n', run_start)
        run = text[run_start + len('Runtime:') : run_end].replace('\n', '').replace('  ', '').strip()
        runs.append(run)
    else:
        run = np.nan
        runs.append(run)

    #distributor => 완료
    dist_start = text.find('Distributor:')
    if dist_start != -1:
        dist_end = text.find('\n\n\n\n\n', dist_start)
        dist = text[dist_start + len('Distributor:') : dist_end].replace('\n', '').replace('  ', '').strip()
        dists.append(dist)
    else:
        dist = np.nan
        dists.append(dist)

    #prod co => 완료
    pc_start = text.find('Production Co:')
    if pc_start != -1:
        pc_end = text.find('\n\n\n\n\n', pc_start)
        pc = text[pc_start + len('Production Co:') : pc_end].replace('\n', '').replace('  ', '').strip()
        pcs.append(pc)
    else:
        pc = np.nan
        pcs.append(pc)

    #box office(gross USA) => 완료
    box_start = text.find('Box Office (Gross USA):')
    if box_start != -1:
        box_end = text.find('\n\n\n\n\n', box_start)
        box = text[box_start + len('Box Office (Gross USA):') : box_end].replace('\n', '').replace('  ', '').strip()
        boxs.append(box)
    else:
        box = np.nan
        boxs.append(box)

    return genres, ols, ds, rds, runs, dists, pcs, boxs

In [ ]:
for row in range(len(tomato)):
    slicing(row, genres, ols, ds, rds, runs, dists, pcs, boxs)
genres = genres
ols = ols
ds = ds
rds = rds
runs = runs
dists = dists
pcs = pcs
boxs = boxs

In [ ]:
sliced_df = pd.DataFrame({'Genres': genres, 'Original Language': ols, 'Director': ds, 'Release Date (Theator)': rds,
                          'Runtime': runs, 'Distributor': dists, 'Prod Co.': pcs, 'Box Office (Gross USA)': boxs})
sliced_df

,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA)
0,"Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M
1,"Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M
2,"Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN
3,"Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M
4,"Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M
...,...,...,...,...,...,...,...,...
178,"Horror, Mystery & thriller",English,Shawn Linden,"Dec 18, 2020",1h 33m,IFC Midnight,"Julijette, Particular Crowd, MarVista Entertai...",$32.2K
179,"Action, Adventure, Fantasy",English,Patty Jenkins,"Dec 25, 2020",2h 31m,Warner Bros. Pictures,"The Stone Quarry, Atlas Entertainment, DC Ente...",$46.5M
180,"Western, Adventure, Drama",English,Paul Greengrass,"Dec 25, 2020",1h 58m,Universal Pictures,"Universal Pictures, Perfect World Pictures, Pr...",$12.7M
181,"Comedy, Mystery & thriller",English,Emerald Fennell,"Dec 25, 2020",1h 53m,Focus Features,"FilmNation Entertainment, LuckyChap Entertainm...",$6.4M


In [ ]:
tomato_final = pd.concat([tomato, sliced_df], axis=1)
tomato_final

,titles,info,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA)
0,Star Wars: The Rise of Skywalker,\n\n\nRating:\nPG-13 (Sci-Fi Violence and Acti...,"Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M
1,Jumanji: The Next Level,\n\n\nRating:\nPG-13 (Some Language|Adventure ...,"Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M
2,Little Women,\n\n\nGenre:\n\n \n...,"Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN
3,Frozen II,\n\n\nRating:\nPG (Some Thematic Elements|Acti...,"Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M
4,The Grudge,\n\n\nRating:\nPG-13 (Mature Thematic Material...,"Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M
...,...,...,...,...,...,...,...,...,...,...
178,Hunter Hunter,\n\n\nRating:\nR\n \n\n...,"Horror, Mystery & thriller",English,Shawn Linden,"Dec 18, 2020",1h 33m,IFC Midnight,"Julijette, Particular Crowd, MarVista Entertai...",$32.2K
179,Wonder Woman 1984,\n\n\nRating:\nPG-13 (Violence|Sequences of Ac...,"Action, Adventure, Fantasy",English,Patty Jenkins,"Dec 25, 2020",2h 31m,Warner Bros. Pictures,"The Stone Quarry, Atlas Entertainment, DC Ente...",$46.5M
180,News of the World,\n\n\nRating:\nPG-13 (Some Language|Disturbing...,"Western, Adventure, Drama",English,Paul Greengrass,"Dec 25, 2020",1h 58m,Universal Pictures,"Universal Pictures, Perfect World Pictures, Pr...",$12.7M
181,Promising Young Woman,\n\n\nRating:\nR (Language Throughout|Drug Use...,"Comedy, Mystery & thriller",English,Emerald Fennell,"Dec 25, 2020",1h 53m,Focus Features,"FilmNation Entertainment, LuckyChap Entertainm...",$6.4M


In [ ]:
tomato_final.drop('info', axis=1, inplace=True); tomato_final

,titles,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA)
0,Star Wars: The Rise of Skywalker,"Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M
1,Jumanji: The Next Level,"Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M
2,Little Women,"Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN
3,Frozen II,"Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M
4,The Grudge,"Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M
...,...,...,...,...,...,...,...,...,...
178,Hunter Hunter,"Horror, Mystery & thriller",English,Shawn Linden,"Dec 18, 2020",1h 33m,IFC Midnight,"Julijette, Particular Crowd, MarVista Entertai...",$32.2K
179,Wonder Woman 1984,"Action, Adventure, Fantasy",English,Patty Jenkins,"Dec 25, 2020",2h 31m,Warner Bros. Pictures,"The Stone Quarry, Atlas Entertainment, DC Ente...",$46.5M
180,News of the World,"Western, Adventure, Drama",English,Paul Greengrass,"Dec 25, 2020",1h 58m,Universal Pictures,"Universal Pictures, Perfect World Pictures, Pr...",$12.7M
181,Promising Young Woman,"Comedy, Mystery & thriller",English,Emerald Fennell,"Dec 25, 2020",1h 53m,Focus Features,"FilmNation Entertainment, LuckyChap Entertainm...",$6.4M


In [ ]:
tomato_final.to_csv("C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/토마토_슬라이싱완료.csv", index=False)

In [ ]:
len(movie)

809

In [ ]:
final = pd.merge(movie, tomato_final, on='titles', how='left'); final

,ranks,titles,index,gross,theaters,per_theater,total_gross,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA)
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690","Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981","Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009","Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042","Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715","Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-,All My Life,359,"$23,840",133,$179,"$899,585","Drama, Romance",English,Marc Meyers,"Dec 4, 2020",1h 33m,Universal Pictures,"Broken Road Productions, Universal Pictures, P...",$909.9K
805,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864",Romance,English,Kevin Connor,NaN,1h 26m,NaN,Larry Levinson Productions,NaN
806,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940","Drama, Mystery & thriller",English,Thomas Bezucha,"Nov 6, 2020",1h 54m,Focus Features,Mazur / Kaplan Company,$9.4M
807,-,Archenemy,359,"$11,003",2,"$5,502","$96,828","Action, Mystery & thriller",English,Adam Egypt Mortimer,"Dec 11, 2020",1h 30m,RLJE Films,"Almost Never Films Inc., Metrol Technology, Sp...",$119.4K


In [ ]:
import calendar
def make_url():
    url_list = []

    for month in range(1, 13):
        # 해당 월의 마지막 일자를 구합니다
        last_day = calendar.monthrange(2020, month)[1]

        for date in range(1, last_day + 1):
            url = f"https://www.the-numbers.com/box-office-chart/weekly/2020/{month}/{date}"
            url_list.append(url)

    return url_list

url_list = make_url()

In [ ]:
for row in range(len(final)):
    index = int(final.loc[row, 'index'])
    start_index = url_list[index].find('2020')
    final.loc[row, 'Date'] = url_list[index][start_index:]

final

,ranks,titles,index,gross,theaters,per_theater,total_gross,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA),Date
0,1,Star Wars: The Rise of Skywalker,2,"$46,053,249","4,406","$10,452","$463,110,690","Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M,2020/1/3
1,2,Jumanji: The Next Level,2,"$33,418,576","4,134","$8,084","$243,124,981","Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M,2020/1/3
2,3,Little Women,2,"$19,932,677","3,308","$6,026","$66,381,009","Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN,2020/1/3
3,4,Frozen II,2,"$15,037,678","3,175","$4,736","$453,623,042","Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M,2020/1/3
4,5,The Grudge,2,"$14,406,715","2,642","$5,453","$14,406,715","Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M,2020/1/3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-,All My Life,359,"$23,840",133,$179,"$899,585","Drama, Romance",English,Marc Meyers,"Dec 4, 2020",1h 33m,Universal Pictures,"Broken Road Productions, Universal Pictures, P...",$909.9K,2020/12/25
805,-,Always & Forever,359,"$18,674",2,"$9,337","$129,864",Romance,English,Kevin Connor,NaN,1h 26m,NaN,Larry Levinson Productions,NaN,2020/12/25
806,-,Let Him Go,359,"$12,315",54,$228,"$9,347,940","Drama, Mystery & thriller",English,Thomas Bezucha,"Nov 6, 2020",1h 54m,Focus Features,Mazur / Kaplan Company,$9.4M,2020/12/25
807,-,Archenemy,359,"$11,003",2,"$5,502","$96,828","Action, Mystery & thriller",English,Adam Egypt Mortimer,"Dec 11, 2020",1h 30m,RLJE Films,"Almost Never Films Inc., Metrol Technology, Sp...",$119.4K,2020/12/25


In [ ]:
final.drop('index', axis=1, inplace=True)

In [ ]:
final = final.rename(columns={'ranks': 'Rank', 'titles': 'Title'}); final

,Rank,Title,gross,theaters,per_theater,total_gross,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA),Date
0,1,Star Wars: The Rise of Skywalker,"$46,053,249","4,406","$10,452","$463,110,690","Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M,2020/1/3
1,2,Jumanji: The Next Level,"$33,418,576","4,134","$8,084","$243,124,981","Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M,2020/1/3
2,3,Little Women,"$19,932,677","3,308","$6,026","$66,381,009","Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN,2020/1/3
3,4,Frozen II,"$15,037,678","3,175","$4,736","$453,623,042","Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M,2020/1/3
4,5,The Grudge,"$14,406,715","2,642","$5,453","$14,406,715","Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M,2020/1/3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,-,All My Life,"$23,840",133,$179,"$899,585","Drama, Romance",English,Marc Meyers,"Dec 4, 2020",1h 33m,Universal Pictures,"Broken Road Productions, Universal Pictures, P...",$909.9K,2020/12/25
805,-,Always & Forever,"$18,674",2,"$9,337","$129,864",Romance,English,Kevin Connor,NaN,1h 26m,NaN,Larry Levinson Productions,NaN,2020/12/25
806,-,Let Him Go,"$12,315",54,$228,"$9,347,940","Drama, Mystery & thriller",English,Thomas Bezucha,"Nov 6, 2020",1h 54m,Focus Features,Mazur / Kaplan Company,$9.4M,2020/12/25
807,-,Archenemy,"$11,003",2,"$5,502","$96,828","Action, Mystery & thriller",English,Adam Egypt Mortimer,"Dec 11, 2020",1h 30m,RLJE Films,"Almost Never Films Inc., Metrol Technology, Sp...",$119.4K,2020/12/25


In [ ]:
final.columns

Index(['Rank', 'Title', 'gross', 'theaters', 'per_theater', 'total_gross',
       'Genres', 'Original Language', 'Director', 'Release Date (Theator)',
       'Runtime', 'Distributor', 'Prod Co.', 'Box Office (Gross USA)', 'Date'],
      dtype='object')

In [ ]:
final = final[['Date', 'Rank', 'Title', 'gross', 'theaters', 'per_theater', 'total_gross',
       'Genres', 'Original Language', 'Director', 'Release Date (Theator)',
       'Runtime', 'Distributor', 'Prod Co.', 'Box Office (Gross USA)']]; final

,Date,Rank,Title,gross,theaters,per_theater,total_gross,Genres,Original Language,Director,Release Date (Theator),Runtime,Distributor,Prod Co.,Box Office (Gross USA)
0,2020/1/3,1,Star Wars: The Rise of Skywalker,"$46,053,249","4,406","$10,452","$463,110,690","Sci-fi, Adventure, Action, Fantasy",English,J.J. Abrams,"Dec 20, 2019",2h 22m,Walt Disney,"Lucasfilm Ltd., Bad Robot",$515.2M
1,2020/1/3,2,Jumanji: The Next Level,"$33,418,576","4,134","$8,084","$243,124,981","Adventure, Action, Comedy, Fantasy",English,Jake Kasdan,"Dec 13, 2019",2h 3m,"Columbia Pictures, Sony Pictures Entertainment","Matt Tolmach Productions, The Detective Agency...",$316.8M
2,2020/1/3,3,Little Women,"$19,932,677","3,308","$6,026","$66,381,009","Drama, Musical",English,William Corrigan,NaN,1h 10m,NaN,CBS,NaN
3,2020/1/3,4,Frozen II,"$15,037,678","3,175","$4,736","$453,623,042","Kids & family, Musical, Comedy, Adventure, Ani...",English,"Chris Buck, Jennifer Lee","Nov 22, 2019",1h 43m,Walt Disney,"Walt Disney Pictures, Walt Disney Animation",$477.4M
4,2020/1/3,5,The Grudge,"$14,406,715","2,642","$5,453","$14,406,715","Horror, Mystery & thriller",English,Takashi Shimizu,"Oct 22, 2004",1h 30m,Sony Pictures Entertainment,"Senator International, Ghost House Pictures",$110.2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,2020/12/25,-,All My Life,"$23,840",133,$179,"$899,585","Drama, Romance",English,Marc Meyers,"Dec 4, 2020",1h 33m,Universal Pictures,"Broken Road Productions, Universal Pictures, P...",$909.9K
805,2020/12/25,-,Always & Forever,"$18,674",2,"$9,337","$129,864",Romance,English,Kevin Connor,NaN,1h 26m,NaN,Larry Levinson Productions,NaN
806,2020/12/25,-,Let Him Go,"$12,315",54,$228,"$9,347,940","Drama, Mystery & thriller",English,Thomas Bezucha,"Nov 6, 2020",1h 54m,Focus Features,Mazur / Kaplan Company,$9.4M
807,2020/12/25,-,Archenemy,"$11,003",2,"$5,502","$96,828","Action, Mystery & thriller",English,Adam Egypt Mortimer,"Dec 11, 2020",1h 30m,RLJE Films,"Almost Never Films Inc., Metrol Technology, Sp...",$119.4K


In [ ]:
final.to_csv("C:/Users/이세은/Desktop/BADA/프로젝트/11월첫째주과제/영화크롤링_2020_이세은_제출용.csv", index=False)

In [ ]:
pcs

['Lucasfilm Ltd., Bad Robot',
 'Matt Tolmach Productions, The Detective Agency, Seven Bucks Productions',
 'CBS',
 'Walt Disney Animation, Walt Disney Pictures',
 'Senator International, Ghost House Pictures',
 'Twentieth Century Fox Animation, Chernin Entertainment, Blue Sky Studios',
 'FilmNation Entertainment, Media Rights Capital',
 'Elara Pictures, Sikelia Productions, Scott Rudin Productions',
 nan,
 nan,
 'Canadian Film or Video Production Tax Credit (CPTC), Appian Way, 75 Year Plan Productions, Misher Films',
 'Chernin Entertainment, 20th Century Fox',
 'Big Beach, TriStar Pictures',
 nan,
 nan,
 'Neal Street Productions, Amblin Entertainment',
 'Bullet Films, Shanghai Bona Cultural Media, Mandarin Motion Pictures',
 'STX Entertainment, MWM',
 'Unison Films',
 'Elizabeth Bay Productions, Studio Babelsberg',
 'Victor Zuev Productions',
 nan,
 'The Mirisch Corporation [us], Universal Pictures',
 '2929 Productions, Victor Solnicki Productions, Copper Heart Entertainment',
 'Focus 

In [ ]:
dists

['Walt Disney',
 'Columbia Pictures, Sony Pictures Entertainment',
 nan,
 'Walt Disney',
 'Sony Pictures Entertainment',
 '20th Century Fox',
 'Lionsgate Films',
 'A24',
 nan,
 nan,
 'Warner Bros. Pictures',
 '20th Century Fox',
 'TriStar Pictures, Sony Pictures Entertainment',
 'Universal Pictures',
 nan,
 'Universal Pictures',
 'Well Go USA',
 'STX Films',
 'Fox Searchlight',
 nan,
 nan,
 nan,
 'Universal Pictures',
 'Metro-Goldwyn-Mayer',
 'Focus Features',
 nan,
 nan,
 'Walt Disney',
 'Warner Bros. Pictures',
 'Self Distributed',
 'Atlas Distribution Company',
 'Sony Pictures Classics',
 'Warner Bros. Pictures',
 'Sony Pictures Classics',
 'Sony Pictures Entertainment, Columbia Pictures',
 'BritBox',
 'Magnolia Pictures',
 'CJ Entertainment',
 'Neon',
 'Amazon Studios',
 'Amazon Studios',
 nan,
 'Columbia Pictures, Sony Pictures Entertainment',
 'A24',
 'Warner Bros. Pictures',
 'Sony Pictures Entertainment, Columbia Pictures',
 nan,
 nan,
 nan,
 'LD Entertainment, Roadside Attract

In [ ]:
runs

['2h 22m',
 '2h 3m',
 '1h 10m',
 '1h 43m',
 '1h 30m',
 '1h 42m',
 '2h 10m',
 '2h 15m',
 '1h 36m',
 '2h 0m',
 '2h 11m',
 '2h 32m',
 '1h 47m',
 '2h 12m',
 '1h 30m',
 '1h 59m',
 '1h 45m',
 '1h 39m',
 '1h 48m',
 '2h 52m',
 '1h 34m',
 nan,
 '2h 12m',
 '1h 30m',
 '2h 5m',
 '1h 28m',
 '1h 30m',
 '1h 58m',
 '2h 16m',
 '1h 21m',
 '1h 35m',
 '1h 53m',
 '1h 49m',
 '1h 53m',
 '1h 39m',
 '2h 18m',
 '1h 33m',
 '2h 8m',
 '1h 53m',
 '2h 19m',
 '1h 33m',
 '1h 35m',
 '1h 58m',
 '2h 15m',
 '2h 31m',
 '2h 39m',
 '1h 30m',
 nan,
 '1h 42m',
 '1h 58m',
 '1h 23m',
 '1h 39m',
 '1h 35m',
 '2h 3m',
 '2h 50m',
 '1h 21m',
 '1h 41m',
 '1h 35m',
 '1h 56m',
 '1h 27m',
 '1h 11m',
 '1h 51m',
 '1h 23m',
 '1h 50m',
 '1h 36m',
 '1h 34m',
 '1h 28m',
 '2h 14m',
 '1h 25m',
 '1h 20m',
 '1h 43m',
 '2h 17m',
 '1h 23m',
 '1h 37m',
 '1h 51m',
 '2h 2m',
 '1h 35m',
 '1h 42m',
 '1h 59m',
 '2h 8m',
 '1h 45m',
 '1h 22m',
 '1h 36m',
 '1h 44m',
 '1h 33m',
 '2h 4m',
 '1h 40m',
 '1h 49m',
 '7m',
 '1h 43m',
 '1h 24m',
 '1h 12m',
 nan,
 '1h

In [ ]:
rds

['Dec 20, 2019',
 'Dec 13, 2019',
 nan,
 'Nov 22, 2019',
 'Oct 22, 2004',
 'Dec 25, 2019',
 'Nov 27, 2019',
 'Dec 25, 2019',
 nan,
 nan,
 'Dec 13, 2019',
 'Nov 15, 2019',
 'Nov 22, 2019',
 'Nov 27, 2019',
 nan,
 'Jan 10, 2020',
 'Dec 25, 2019',
 'Nov 22, 2019',
 'Nov 8, 2019',
 nan,
 nan,
 nan,
 'Jun 18, 1976',
 'Dec 25, 2006',
 'Nov 1, 2019',
 nan,
 nan,
 'Oct 18, 2019',
 'Jan 10, 2020',
 'Oct 11, 2019',
 'Oct 25, 2019',
 'Oct 4, 2019',
 'Nov 15, 2019',
 'Dec 25, 2019',
 'Oct 18, 2019',
 'Nov 27, 2019',
 'Dec 13, 2019',
 'Dec 20, 2019',
 'Dec 27, 2019',
 'Dec 20, 2019',
 'Nov 8, 2019',
 nan,
 'Nov 15, 2019',
 'Nov 15, 2019',
 'Nov 8, 2019',
 'Jul 26, 2019',
 nan,
 nan,
 nan,
 'Sep 27, 2019',
 'Sep 6, 1952',
 'Dec 6, 2019',
 'Nov 1, 2019',
 'Oct 25, 2019',
 nan,
 'Dec 6, 2019',
 'Nov 8, 2019',
 'Sep 6, 2019',
 'Nov 22, 2019',
 'Nov 15, 2019',
 'Jul 13, 2001',
 'Sep 20, 2019',
 'Oct 4, 2019',
 'Sep 13, 2019',
 'Jan 19, 1990',
 'Dec 6, 2019',
 'Nov 13, 2019',
 'Oct 30, 2019',
 'Oct 16, 2

In [ ]:
ols

['English',
 'English',
 'English',
 'English',
 'English',
 'English',
 'English',
 'English',
 'English',
 'English (United Kingdom)',
 'English',
 'English',
 'English',
 'English',
 'English',
 'English',
 'Chinese',
 'English',
 'English',
 'English',
 'English',
 nan,
 'English',
 'English',
 'English',
 'English',
 'Russian',
 'English',
 'English',
 'English',
 'English',
 'Spanish (Spain)',
 'English',
 'English',
 'English',
 'English (United Kingdom)',
 'English',
 'Korean',
 'English',
 'Portuguese (Brazil)',
 'English',
 'English',
 'English',
 'English',
 'English',
 'English',
 'French (France)',
 nan,
 'English',
 'English',
 'Italian',
 'English',
 'English',
 'French (France)',
 'English (United Kingdom)',
 'Spanish',
 'English',
 'English',
 'French (France)',
 'English',
 'English',
 'Japanese',
 'English',
 'English',
 'English',
 'Italian',
 'English',
 'English',
 'English',
 'English',
 'English',
 'French (France)',
 'English',
 'English',
 'Japanese',
 'Hindi'

In [ ]:
genres

['Sci-fi, Adventure, Action, Fantasy',
 'Adventure, Action, Comedy, Fantasy',
 'Drama, Musical',
 'Kids & family, Musical, Comedy, Adventure, Animation',
 'Horror, Mystery & thriller',
 'Kids & family, Comedy, Adventure, Action, Animation',
 'Mystery & thriller, Comedy, Drama',
 'Mystery & thriller, Comedy',
 'Sci-fi',
 'Musical',
 'Drama',
 'History, Drama, Biography',
 'Biography, Drama',
 'Crime, Drama, Romance',
 'Horror, Sci-fi',
 'War, History, Drama',
 'Action',
 'Action, Mystery & thriller, Crime, Drama',
 'Comedy, Drama',
 'Drama',
 'Horror',
 'Drama',
 'War',
 'Holiday, Horror',
 'Biography, History, Drama',
 'Mystery & thriller',
 'Action, Adventure',
 'Fantasy, Adventure',
 'Drama',
 'Documentary',
 'Documentary',
 'Drama',
 'Mystery & thriller, Crime, Drama',
 'Drama, Mystery & thriller',
 'Comedy, Horror, Action',
 'Documentary',
 'Documentary, Biography, Music',
 'Action, Drama',
 'Drama',
 'Drama, Romance',
 'Drama',
 'Horror',
 'Action, Adventure, Comedy',
 'Drama',
 '

In [ ]:
ds

['J.J. Abrams',
 'Jake Kasdan',
 'William Corrigan',
 'Chris Buck, Jennifer Lee',
 'Takashi Shimizu',
 'Troy Quane, Nick Bruno',
 'Rian Johnson',
 'Benny Safdie, Josh Safdie',
 'Paul Wynne',
 'David Mallet',
 'Clint Eastwood',
 'James Mangold',
 'Marielle Heller',
 'Melina Matsoukas',
 nan,
 'Sam Mendes',
 'Wilson Yip',
 'Brian Kirk',
 'Taika Waititi',
 'Terrence Malick',
 'Mariano Baino',
 nan,
 'Jack Smight',
 'Glen Morgan',
 'Kasi Lemmons',
 'David DeCoteau',
 'Yuri Kuzmenko',
 'Joachim Rønning',
 'Destin Daniel Cretton',
 'Louie Schwartzberg',
 'Justin Folk',
 'Pedro Almodóvar',
 'Bill Condon',
 'François Girard',
 'Ruben Fleischer',
 'Michael Apted',
 'Alla Kovgan',
 'Kim Byeong-seo, Lee Hae-jun',
 'Chinonye Chukwu',
 'Karim Aïnouz',
 "Alma Har'el",
 'Simon Hunter',
 'Elizabeth Banks',
 'Trey Edward Shults',
 'Mike Flanagan',
 'Quentin Tarantino',
 'Nicolas Herdt',
 nan,
 'Leslie Megahey',
 'Rupert Goold',
 'Federico Fellini',
 'Lino DiSalvo',
 'Errol Morris',
 'Nadav Lapid',
 'Ja

In [ ]:
len(ols)

343